# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3e6851ccd0>
├── 'a' --> tensor([[-1.5924, -1.1764, -0.0293],
│                   [-0.0476,  0.0546, -1.0612]])
└── 'x' --> <FastTreeValue 0x7f3e6851cdc0>
    └── 'c' --> tensor([[-0.3609, -0.6039, -1.8249,  0.8903],
                        [ 0.9392,  0.7800, -0.7008, -1.4694],
                        [-0.3833, -1.6897, -1.2121, -0.3682]])

In [4]:
t.a

tensor([[-1.5924, -1.1764, -0.0293],
        [-0.0476,  0.0546, -1.0612]])

In [5]:
%timeit t.a

63.3 ns ± 0.0509 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3e6851ccd0>
├── 'a' --> tensor([[-1.9698,  0.1596,  0.0916],
│                   [ 1.1979,  1.4754,  0.3535]])
└── 'x' --> <FastTreeValue 0x7f3e6851cdc0>
    └── 'c' --> tensor([[-0.3609, -0.6039, -1.8249,  0.8903],
                        [ 0.9392,  0.7800, -0.7008, -1.4694],
                        [-0.3833, -1.6897, -1.2121, -0.3682]])

In [7]:
%timeit t.a = new_value

61.2 ns ± 0.0297 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.5924, -1.1764, -0.0293],
               [-0.0476,  0.0546, -1.0612]]),
    x: Batch(
           c: tensor([[-0.3609, -0.6039, -1.8249,  0.8903],
                      [ 0.9392,  0.7800, -0.7008, -1.4694],
                      [-0.3833, -1.6897, -1.2121, -0.3682]]),
       ),
)

In [10]:
b.a

tensor([[-1.5924, -1.1764, -0.0293],
        [-0.0476,  0.0546, -1.0612]])

In [11]:
%timeit b.a

58 ns ± 0.0215 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7050,  0.9053, -0.2145],
               [ 1.0503,  1.2144,  0.2178]]),
    x: Batch(
           c: tensor([[-0.3609, -0.6039, -1.8249,  0.8903],
                      [ 0.9392,  0.7800, -0.7008, -1.4694],
                      [-0.3833, -1.6897, -1.2121, -0.3682]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.115 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

851 ns ± 0.183 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 16.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 525 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 624 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3d8fe38d60>
├── 'a' --> tensor([[[-1.5924, -1.1764, -0.0293],
│                    [-0.0476,  0.0546, -1.0612]],
│           
│                   [[-1.5924, -1.1764, -0.0293],
│                    [-0.0476,  0.0546, -1.0612]],
│           
│                   [[-1.5924, -1.1764, -0.0293],
│                    [-0.0476,  0.0546, -1.0612]],
│           
│                   [[-1.5924, -1.1764, -0.0293],
│                    [-0.0476,  0.0546, -1.0612]],
│           
│                   [[-1.5924, -1.1764, -0.0293],
│                    [-0.0476,  0.0546, -1.0612]],
│           
│                   [[-1.5924, -1.1764, -0.0293],
│                    [-0.0476,  0.0546, -1.0612]],
│           
│                   [[-1.5924, -1.1764, -0.0293],
│                    [-0.0476,  0.0546, -1.0612]],
│           
│                   [[-1.5924, -1.1764, -0.0293],
│                    [-0.0476,  0.0546, -1.0612]]])
└── 'x' --> <FastTreeValue 0x7f3da2a6bd60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 53.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3d8fe08df0>
├── 'a' --> tensor([[-1.5924, -1.1764, -0.0293],
│                   [-0.0476,  0.0546, -1.0612],
│                   [-1.5924, -1.1764, -0.0293],
│                   [-0.0476,  0.0546, -1.0612],
│                   [-1.5924, -1.1764, -0.0293],
│                   [-0.0476,  0.0546, -1.0612],
│                   [-1.5924, -1.1764, -0.0293],
│                   [-0.0476,  0.0546, -1.0612],
│                   [-1.5924, -1.1764, -0.0293],
│                   [-0.0476,  0.0546, -1.0612],
│                   [-1.5924, -1.1764, -0.0293],
│                   [-0.0476,  0.0546, -1.0612],
│                   [-1.5924, -1.1764, -0.0293],
│                   [-0.0476,  0.0546, -1.0612],
│                   [-1.5924, -1.1764, -0.0293],
│                   [-0.0476,  0.0546, -1.0612]])
└── 'x' --> <FastTreeValue 0x7f3da2881040>
    └── 'c' --> tensor([[-0.3609, -0.6039, -1.8249,  0.8903],
                        [ 0.9392,  0.7800, -0.7008, -1.4694],
                 

In [23]:
%timeit t_cat(trees)

31.5 µs ± 86.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.5 µs ± 70 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.5924, -1.1764, -0.0293],
                [-0.0476,  0.0546, -1.0612]],
       
               [[-1.5924, -1.1764, -0.0293],
                [-0.0476,  0.0546, -1.0612]],
       
               [[-1.5924, -1.1764, -0.0293],
                [-0.0476,  0.0546, -1.0612]],
       
               [[-1.5924, -1.1764, -0.0293],
                [-0.0476,  0.0546, -1.0612]],
       
               [[-1.5924, -1.1764, -0.0293],
                [-0.0476,  0.0546, -1.0612]],
       
               [[-1.5924, -1.1764, -0.0293],
                [-0.0476,  0.0546, -1.0612]],
       
               [[-1.5924, -1.1764, -0.0293],
                [-0.0476,  0.0546, -1.0612]],
       
               [[-1.5924, -1.1764, -0.0293],
                [-0.0476,  0.0546, -1.0612]]]),
    x: Batch(
           c: tensor([[[-0.3609, -0.6039, -1.8249,  0.8903],
                       [ 0.9392,  0.7800, -0.7008, -1.4694],
                       [-0.3833, -1.6897, -1.2121, -0.3682]],
         

In [26]:
%timeit Batch.stack(batches)

81.6 µs ± 2.51 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.5924, -1.1764, -0.0293],
               [-0.0476,  0.0546, -1.0612],
               [-1.5924, -1.1764, -0.0293],
               [-0.0476,  0.0546, -1.0612],
               [-1.5924, -1.1764, -0.0293],
               [-0.0476,  0.0546, -1.0612],
               [-1.5924, -1.1764, -0.0293],
               [-0.0476,  0.0546, -1.0612],
               [-1.5924, -1.1764, -0.0293],
               [-0.0476,  0.0546, -1.0612],
               [-1.5924, -1.1764, -0.0293],
               [-0.0476,  0.0546, -1.0612],
               [-1.5924, -1.1764, -0.0293],
               [-0.0476,  0.0546, -1.0612],
               [-1.5924, -1.1764, -0.0293],
               [-0.0476,  0.0546, -1.0612]]),
    x: Batch(
           c: tensor([[-0.3609, -0.6039, -1.8249,  0.8903],
                      [ 0.9392,  0.7800, -0.7008, -1.4694],
                      [-0.3833, -1.6897, -1.2121, -0.3682],
                      [-0.3609, -0.6039, -1.8249,  0.8903],
                      [ 0.9392,  

In [28]:
%timeit Batch.cat(batches)

164 µs ± 351 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

327 µs ± 4.76 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
